<a href="https://colab.research.google.com/github/Tejasmallikarjuncj/kmean/blob/main/kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import files